In [ ]:
! pip install sentence_transformers

In [4]:
import pandas as pd
import numpy as np

import json
import re

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances

from nltk.corpus import stopwords
from sentence_transformers import SentenceTransformer, util
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
! pip install transformers

In [6]:
from sentence_transformers import SentenceTransformer, util
import numpy as np

In [ ]:
model = SentenceTransformer('stsb-roberta-large')

In [ ]:
with open("/content/drive/MyDrive/cs48k/issues_final.json", 'r') as clean_json_file:
    data = json.load(clean_json_file)
    issues_df=pd.DataFrame(data, columns=['id', 'title', 'body', 'comments', 'originalId', 'originalTitle', 'originalBody'])

    stop_words_l=stopwords.words('english')

    issues_df['title_cleaned']=issues_df.title.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z0-9]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )
    issues_df['body_cleaned']=issues_df.body.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z0-9]',' ',w).lower() for w in str(x).split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )
    issues_df['original_title_cleaned']=issues_df.originalTitle.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z0-9]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )
    issues_df['original_body_cleaned']=issues_df.originalBody.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z0-9]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stop_words_l) )

    print(issues_df.head())

In [ ]:
title_list = list(issues_df['title_cleaned'])
print(title_list)
print(len(title_list))
title_list = title_list[0:40]
print(len(title_list))



In [33]:
# encode list of sentences to get their embeddings
embedding1 = model.encode(title_list, convert_to_tensor=True)
embedding2 = model.encode(title_list, convert_to_tensor=True)# compute similarity scores of two embeddings
cosine_scores = util.pytorch_cos_sim(embedding1, embedding2)



In [39]:
similars_map = {}
for i in range(len(title_list)):
    # print("issue id:", issues_df.iloc[i]["id"], " Issue Title:", title_list[i])
    # print("Similar Issues:\n")
    similars_map[(issues_df.iloc[i]["id"], issues_df.iloc[i]["title"])] = []
    for j in range(len(title_list)):
        cosine_similarity = cosine_scores[i][j].item()
        if cosine_similarity > 0.6 and i != j:
            similars_map[(issues_df.iloc[i]["id"], issues_df.iloc[i]["title"])].append((issues_df.iloc[j], cosine_similarity))
            # print("issue id:", issues_df.iloc[j]["id"], " Issue Title:", title_list[j] )
            # print("Similarity Score:", cosine_scores[i][j].item())
            # print("\n")
    


In [41]:
for k, v in similars_map.items():
  if len(v) > 0:
    print("* issue id:", k[0], " Issue Title:", k[1])
    print("  Similar Issues:")
    i = 1
    for issue, cosine in v:
       print("   ", i , "- Similarity Score:", cosine, " ->  issue id:", issue["id"], " Issue Title:", issue["title"])
       i+= 1
    print("\n")



* issue id: 102068  Issue Title: [tool_crash] NoSuchMethodError: NoSuchMethodError: The getter 'assetPathsToEvict' was called on null.Receiver: nullTried calling: assetPathsToEvict
  Similar Issues:
    1 - Similarity Score: 0.9999991655349731  ->  issue id: 101606  Issue Title: [tool_crash] NoSuchMethodError: NoSuchMethodError: The getter 'assetPathsToEvict' was called on null.Receiver: nullTried calling: assetPathsToEvict


* issue id: 101975  Issue Title: [tool_crash] ArgumentError: Invalid argument(s): Cannot find executable for /Users/wangyao/development/flutter/bin/cache/artifacts/engine/android-x64-release/darwin-x64/gen_snapshot.
  Similar Issues:
    1 - Similarity Score: 0.6019042730331421  ->  issue id: 101764  Issue Title: [tool_crash] FileSystemException: Failed to decode data using encoding 'utf-8', null
    2 - Similarity Score: 0.6420588493347168  ->  issue id: 101714  Issue Title: [tool_crash] FileSystemException: Cannot create link, OS Error: Incorrect function., errn